In [1]:
import pandas as pd
from utils.db import source_engine as engine

query = """
SELECT *
FROM "tWorkOrders" 
WHERE "RequestCreateDatetime" >= date_trunc('MONTH', CURRENT_DATE - INTERVAL '1 MONTH')
    AND "RequestCreateDatetime" < date_trunc('MONTH', CURRENT_DATE)
"""

# Execute the query and load the data into a DataFrame
last_month_work_orders = pd.read_sql_query(query, engine)

In [2]:
import pandas as pd
from utils.db import source_engine as engine

with open('sql/WorkOrderConversations.sql', 'r') as file:
    work_order_conversations = file.read()

# Replace the placeholder with the actual work order ID
work_order_ids = last_month_work_orders["Id"][:11].to_list()
work_order_ids_text = ','.join(map(lambda x: str(x),work_order_ids))
query = work_order_conversations.format(WorkOrderIds=work_order_ids_text) 

# Execute the query and load the data into a DataFrame
df = pd.read_sql_query(query, engine)

In [3]:
len(df[:])

379

In [8]:
# Import necessary functions from the data_processings.chat_messages_processing module
from data_processings.chat_messages_processing import add_event_description, parse_message_content

# Apply the parse_message_content function to each row in the DataFrame to create a new column 'CleanedText'
df['CleanedText'] = df.apply(parse_message_content, axis=1)

# Apply the add_event_description function to each row in the DataFrame to create a new column 'EventDescription'
df['EventDescription'] = df.apply(add_event_description, axis=1)

In [9]:
df[df["WorkOrderId"]==work_order_ids[0]][['CleanedText','EventDescription','SendDatetime','SenderRoleName']]

,CleanedText,EventDescription,SendDatetime,SenderRoleName
110,New work order 'REDBIRDGPM-2390-1' was created.,New work order created,2025-01-23 17:20:08.189795,SuperAdmin
114,A new Repair work order has been created.,Work order classification changed,2025-01-23 17:47:15.243660,Assistant
115,"Hi Katherine, I’m Paulina with Red Bird GA PM....",SMS to Resident,2025-01-23 17:49:03.650369,Assistant
116,New status: Escalation\nOther Issue: EMC LEVEL 2,Status Update,2025-01-23 17:49:35.440631,Assistant
126,New status: Awaiting Tenant Response\nThe tena...,Status Update,2025-01-23 18:25:28.001083,Assistant
127,New status: Escalated\nThe tenant stated that ...,Status Update,2025-01-23 22:12:13.353440,Assistant
128,New status: Escalation\nOther Issue: Please as...,Status Update,2025-01-23 22:12:47.943396,Assistant
132,**January 24**\n\n**Update:** We have sent a m...,Status Update,2025-01-24 15:52:36.456889,Assistant
150,Next action date updates to: 01/28/2025 16:55:00,Next action date updated,2025-01-24 21:55:12.045246,Assistant
151,Next action date updates to: 01/28/2025 00:00:00,Next action date updated,2025-01-24 22:23:00.664041,Assistant


In [10]:
wo = df[df["WorkOrderId"]==work_order_ids[0]][['CleanedText','EventDescription','SendDatetime','SenderRoleName']]

In [11]:
wo.to_csv("data/wo.csv", index=False)

In [ ]:
# TODO: Visualize work order conversations in timelines:
# Done: utils/timeline.py
# Usage:
# streamlit run utils/timeline.py data/wo.csv